<a href="https://colab.research.google.com/github/kenjan1974/Prediction-of-quality-of-Wine/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C06_%E7%B4%85%E9%85%92%E5%93%81%E8%B3%AA%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# 1. 讀取原始資料
df = pd.read_csv(
    'https://raw.githubusercontent.com/aniruddhachoudhury/'
    'Red-Wine-Quality/master/winequality-red.csv'
)

# 2. 處理重複值
df = df.drop_duplicates()

# 3. 處理缺失值（本資料無缺失，但一般流程會這麼做）
df = df.dropna()

# 4. 重新打亂順序，並重設索引
df_train = df.sample(frac=1, random_state=42).reset_index(drop=True)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df_train.drop('quality', axis = 1)
y = df_train['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
rfc = RandomForestClassifier(n_estimators=200)
rfc_eval = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 5)
print(rfc_eval)

[0.57798165 0.63302752 0.61290323 0.62211982 0.59447005]


2. **欄位定義**（依照 Kaggle/UCI 文件）

   > 所有單位都以 g/dm³ 計，除非另註。

   * `fixed_acidity`：固定酸度（檸檬酸、蘋果酸等除揮發性酸外的總酸）
   * `volatile_acidity`：揮發性酸度（主要是醋酸，過高會產生醋味）
   * `citric_acid`：檸檬酸（少量有助於風味和新鮮感）
   * `residual_sugar`：殘留糖分（發酵後仍存在的糖，過高易甜膩）
   * `chlorides`：氯化物（鹽分量，過高會感覺鹹）
   * `free_sulfur_dioxide`：游離二氧化硫（抗氧化劑，過量會嗆鼻）
   * `total_sulfur_dioxide`：總二氧化硫（游離＋結合型）
   * `density`：密度（與糖分、酒精、溫度有關）
   * `pH`：酸鹼值（小於3 表示偏酸）
   * `sulphates`：硫酸鹽（促進防腐和風味）
   * `alcohol`：酒精含量（% v/v）
   * `quality`：品質分數（整數，0–10 分，主觀評分）

3. **特徵（features）分佈觀察**

   * **固定酸度 (fixed\_acidity)**：大多集中在 6–10；平均約 8.3，略偏右尾。

   * **揮發性酸度 (volatile\_acidity)**：多數落在0.2–0.6；平均約0.53，右偏（少數高醋酸樣本）。

   * **檸檬酸 (citric\_acid)**：大部分在0.0–0.6；平均約0.27，零或極低的比例也不少。

   * **殘留糖分 (residual\_sugar)**：多集中在0.9–2.5，平均約2；明顯右偏（最高可達15以上）。

   * **氯化物 (chlorides)**：主要落在0.03–0.10，平均約0.087，分佈窄且左偏。

   * **游離二氧化硫 (free\_sulfur\_dioxide)**：多在0–30 mg/dm³，平均約15，很長尾（最高上百）。

   * **總二氧化硫 (total\_sulfur\_dioxide)**：多在50–150 mg/dm³，平均約120，同樣長尾。

   * **密度 (density)**：範圍極窄，約0.990–1.004 g/cm³，分佈近常態，平均≈0.996。

   * **pH**：集中在3.1–3.6，平均約3.31，分佈略偏左。

   * **硫酸鹽 (sulphates)**：主要落在0.4–0.8，平均約0.66，分佈偏左。

   * **酒精含量 (alcohol)**：範圍7–14%，平均約10.4%，分佈近常態但略右尾。

   > **總結**：多數特徵的分佈都呈現輕微偏態（尤其是殘糖、揮發性酸度、二氧化硫），而像密度、pH、酒精含量則較接近對稱分佈。

4. **標籤（quality）分佈**

   * 質量分數範圍：3–8 分

   * **5 分與 6 分** 最常見，合計佔全體樣本約70%

   * 3、4、7、8 分樣本相對稀少，其中 8 分極少（約0.3%）

   > **小結**：資料標籤明顯偏向中間分數，屬於高度不平衡的多分類問題。


1. **資料分析工作流程必備環節（不考慮準確度）**

   1. **定義問題**：釐清業務或研究目標，決定要做分類、迴歸或其他分析。
   2. **資料蒐集**：取得原始資料（CSV、資料庫、API…）。
   3. **資料檢視（Exploration）**：快速檢查欄位型別、缺失值、重複值與大致分佈。
   4. **資料清理**：處理缺失值、重複值、異常值，統一欄位名稱、資料格式。
   5. **特徵轉換**：必要時做類別編碼、數值標準化或拆／合新特徵。
   6. **資料切分**：依需求切成訓練集、測試集（或驗證集）。
   7. **Baseline 建立**：選個簡單模型／基準模型跑一次，檢查 pipeline 無誤。
   8. **模型訓練與預測**：用訓練集訓練、測試集預測。
   9. **評估與報告**：算指標、畫圖、撰寫結論。